## Chapter 4 Demo - PINN: Physics Informed Neural Network

In [ ]:
import torch
from torch.autograd import grad
import matplotlib.pyplot as plt
from torch.optim import Adam

## 1D

In [ ]:
# wrapped automatic differentiation computes derivatives
def getDerivative(y, x):
    dydx = grad(y, x, torch.ones_like(y),     create_graph=True,     retain_graph=True)[0]
    return dydx

# residual equation
def r(model, x, EA, p):
    u = model(x)
    dudx = getDerivative(u, x)
    dEAdudxx = getDerivative(EA(x) * dudx, x)
    r = dEAdudxx + p(x)
    return r

In [ ]:
inputDim = outputDim = 1
hiddenDim = 100

model = torch.nn.Sequential(torch.nn.Linear(inputDim, hiddenDim),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiddenDim, outputDim))

uPred = model(torch.tensor([0.5]))
print(uPred)

In [ ]:
x = torch.linspace(0, 1, 10, requires_grad=True).unsqueeze(1)
EA = lambda x: 1 + 0 * x
p = lambda x: 4 * torch.pi ** 2 * torch.sin(2 * torch.pi * x)
rPred = r(model, x, EA, p)
lossR = torch.sum(rPred ** 2)
 
print(f"Untrained Model total loss, lossR: {lossR}\n")

table = torch.cat((x, rPred), dim=1).detach().numpy()
print(f"Untrained Model prediction for\n      x      \t  rPred: \n{table}")

plt.plot(x.detach().numpy(),  rPred.detach().numpy())
plt.xlabel('x')
plt.ylabel('rPred')

In [ ]:
# Impose Dirichlet BCs with Loss function at Boundaries
u0 = 0
u1 = 0

u0Pred = model(torch.tensor([0.]))
u1Pred = model(torch.tensor([1.]))
lossB = (u0Pred - u0) ** 2 + (u1Pred - u1) ** 2
print(f"lossB: {lossB.item()}")

## 2D
**todo** this section in the book and below don't work directly as written in text: need to define a 2D model and 2D derivative. I do the model in the cell below, but want to check the derivative - unclear exactly what the syntax `dsig11dx1 = getDerivative(stress[0], x1, 1)` should do.

In [ ]:
inputDim = 2
outputDim = 1
hiddenDim = 100

model = torch.nn.Sequential(torch.nn.Linear(inputDim, hiddenDim),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiddenDim, outputDim))

In [ ]:
Lx1 = Lx2= 1
Nx1 = Nx2 = 10

x1 = torch.linspace(0, Lx1, Nx1)
x2 = torch.linspace(0, Lx2, Nx2)
x1, x2 = torch.meshgrid(x1, x2, indexing='ij')
x1.requires_grad = True
x2.requires_grad = True
modelInput = torch.cat((x1.reshape(-1, 1), x2.reshape(-1, 1)), 1)

uPred = model(modelInput)
print(uPred)

In [ ]:
def getStrains(u, x1, x2, Nx1, Nx2):
    strain = torch.zeros((3, Nx1, Nx2))
    strain[0] = getDerivative(u[:, 0].reshape(Nx1, Nx2), x1, 1)
    strain[1] = getDerivative(u[:, 1].reshape(Nx1, Nx2), x2, 1)
    strain[2] = 0.5 * (getDerivative(u[:, 0].reshape(Nx1, Nx2), x2, 1)
                     + getDerivative(u[:, 1].reshape(Nx1, Nx2), x1, 1))
    return strain

def getDerivative(y, x):
    dydx = grad(y, x, torch.ones_like(y),     create_graph=True,     retain_graph=True)[0]
    return dydx

In [ ]:
strain = getStrains(uPred, x1, x2, Nx1, Nx2)
stress = torch.tensordot(C, strain, dims=1)

dsig11dx1 = getDerivative(stress[0], x1, 1)
dsig12dx2 = getDerivative(stress[2], x2, 1)
dsig21dx1 = getDerivative(stress[2], x1, 1)
dsig22dx2 = getDerivative(stress[1], x2, 1)

lossPDE = 0  # equilibrium computation
lossPDE += (torch.sum((dsig11dx1 + dsig12dx2 + p1(x1, x2)) ** 2)
           / (Nx1 * Nx2))  # divide by number of collocation points
lossPDE += (torch.sum((dsig21dx1 + dsig22dx2 + p2(x1, x2)) ** 2)
           / (Nx1 * Nx2))  # divide by number of collocation points
